In [1]:
import os

In [2]:
%pwd

'c:\\Users\\ranro\\Documents\\ML_Projects\\end_to_end_ml_mlflow\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\ranro\\Documents\\ML_Projects\\end_to_end_ml_mlflow'

In [5]:
# defining environement variables
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/ranro3/end_to_end_ml_mlflow.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="ranro3"
os.environ["MLFLOW_TRACKING_PASSWORD"]="4841ebe9c6cccbf89d4e5c07339d949acf33652e"

In [18]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path  
    target_column: str
    mlflow_uri: str

In [19]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_dirs, save_to_json

In [20]:
class ConfigurationManager:
    """
    Class to represent a configuration managder for initialize
    configurations from config.yaml.
    """
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        # creates directories for model_trainer
        create_dirs([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        target = self.schema.TARGET_COLUMN

        create_dirs([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=target.name,
            mlflow_uri="https://dagshub.com/ranro3/end_to_end_ml_mlflow.mlflow"
        )

        return model_evaluation_config
    

In [21]:
import os
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [22]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, preds):
        """
        Calculates and returns the following metrics:
            - Mean Squared Error
            - Mean Absolute Error
            - R^2

        Args:
            actual - contains the true values
            pred - contains the predicted values from the model

        Returns:
            rmse - double, root mean squeared error
            mae - double, mean absolute error
            r2 - double, r^2 metric
        """
        rmse = np.sqrt(mean_squared_error(actual, preds))
        mae = mean_absolute_error(actual, preds)
        r2 = r2_score(actual, preds)

        return rmse, mae, r2
    
    def log_to_mlflow(self):
        """
        Logs the metrics, parametrs and model to MLFlow
        """
        
        # loads test data and model
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        # defines X_test and y_test
        X_test = test_data.drop([self.config.target_column], axis=1)
        y_test = test_data[[self.config.target_column]]

        # sets tracking uri
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        # starts mlflow tracking
        with mlflow.start_run():
            # calculate scores
            preds = model.predict(X_test)
            (rmse, mae, r2) = self.eval_metrics(y_test, preds)

            # saves metrics locally
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_to_json(path=Path(self.config.metric_file_name), data=scores)

            # logs parameters to mlflow
            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)
            
            # logs model to mlflow
            mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticNet")
           



In [24]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_to_mlflow()
except Exception as e:
    raise e
    

[2023-11-21 11:20:36,623: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-11-21 11:20:36,627: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-21 11:20:36,629: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-11-21 11:20:36,631: INFO: common: Folder artifacts was created successfully]
[2023-11-21 11:20:36,632: INFO: common: Folder artifacts/model_evaluation was created successfully]
[2023-11-21 11:20:39,243: INFO: common: json file saved at artifacts\model_evaluation\metrics.json]


Registered model 'ElasticNet' already exists. Creating a new version of this model...
2023/11/21 11:20:58 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticNet, version 4
Created version '4' of model 'ElasticNet'.
